In [ ]:
%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 17.2 MB/s eta 0:00:00
  Using cached nvidia_cuda

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
# import os
# from dotenv import load_dotenv
# load_dotenv()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import userdata
# GROQ_API_KEY = userdata.get('GROQ_API_KEY')
GROQ_API_KEY = "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"

In [ ]:
# data ingestion
reader = SimpleDirectoryReader(input_files=["/content/amazon.txt"])
documents = reader.load_data()

In [ ]:
len(documents)

1

In [ ]:
documents[0].metadata

{'file_path': '/content/amazon.txt',
 'file_name': 'amazon.txt',
 'file_type': 'text/plain',
 'file_size': 1084,
 'creation_date': '2024-06-21',
 'last_modified_date': '2024-06-21'}

In [ ]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
len(nodes)

5

In [ ]:
nodes[0].metadata

{'file_path': '/content/amazon.txt',
 'file_name': 'amazon.txt',
 'file_type': 'text/plain',
 'file_size': 1084,
 'creation_date': '2024-06-21',
 'last_modified_date': '2024-06-21'}

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

In [ ]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

In [ ]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
vector_index.storage_context.persist(persist_dir="./storage_mini")

In [ ]:
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

In [ ]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine(service_context=service_context)

In [ ]:
query = "What payment methods are accepted at Amazon"
resp = query_engine.query(query)

In [ ]:
print(resp.response)

Cash, Amazon UPI Scan & Pay, Amazon SMS Pay link, credit/debit cards, UPI, and digital payment facilities via UPI/QR Code –Scan & Pay.


In [ ]:
query = "Can you explain about splitting payment between cards ?"
resp = query_engine.query(query)

In [ ]:
print(resp.response)

Based on the provided context, there is no information about splitting payment between cards. The context only discusses various payment methods such as Pay on Delivery, Debit EMI, Down Payment, and Amazon Pay Later, but it does not mention splitting payment between cards.


In [ ]:
query = "I have an international card, will it be accepted ? And how many cards can I add to Amazon"
resp = query_engine.query(query)

In [ ]:
print(resp.response)

According to the provided information, Pay on Delivery is not available for exports/international customers. This implies that international cards may not be accepted for certain payment methods. However, it does not explicitly state that international cards are not accepted altogether.

Regarding the number of cards you can add to Amazon, as per the new RBI guidelines, a card can be saved to a maximum of 10 Amazon accounts. If an existing card is saved to more than 10 accounts, the card will be deleted even if the customer consents to save the card.


In [ ]:
query = "I want to contact customer support for more details. Can you share any link for the same ?"
resp = query_engine.query(query)

In [ ]:
print(resp.response)

Unfortunately, the provided context information does not contain a direct link to contact customer support. However, you can visit the "Help" or "Contact Us" page on the Amazon website, which is not explicitly mentioned in the context, but is a common practice on e-commerce websites.


In [ ]:
query = "Give some personal finance tips"
resp = query_engine.query(query)

In [ ]:
print(resp.response)

Here are some personal finance tips: 

Manage your credit/debit cards wisely, and make sure to remove older credentials to avoid exceeding the maximum limit of 20 cards. Consider using alternative payment methods like Unified Payment Interface (UPI) or Net Banking for a more diversified financial portfolio. Additionally, be mindful of the new RBI guidelines and provide consent for card tokenization to ensure seamless transactions.


In [ ]:
query = "How many max cards can be added to Amazon Account ?"
resp = query_engine.query(query)
print(resp.response)

As per the new RBI guidelines, a card can be saved to a maximum of 10 Amazon accounts.


In [ ]:
query = "I know that a card can be saved to maximum of 10 amazon accounts but how many cards can one single account have ?"
resp = query_engine.query(query)
print(resp.response)

The context information does not explicitly state the maximum number of cards that can be saved to a single Amazon account. It only mentions that a card can be saved to a maximum of 10 Amazon accounts. Therefore, it cannot be determined from the provided context how many cards can be saved to a single account.


In [ ]:
query = "What can I return?"
resp = query_engine.query(query)
print(resp.response)

You can request a return on most items that are clearly marked as 'returnable' on the product detail page and/or in our policy and have a 'return period' period. Products that are damaged, defective, missing parts of the product or not as described on the product details page are eligible for return.


In [ ]:
query = "How do I return a gift card?"
resp = query_engine.query(query)
print(resp.response)

Gift cards once purchased cannot be canceled or refunded due to legal restrictions.
